In [17]:
import imageio
import imgaug as ia
from imgaug.augmentables.kps import Keypoint
from imgaug.augmentables.kps import Keypoint, KeypointsOnImage
import imgaug.augmenters as iaa
import numpy as np
ia.seed(3)
%matplotlib inline

In [18]:
import os
import json
import os.path as osp
from os.path import isdir, join
import re 

In [19]:
dataset_dir = "./hockey_dataset"
SERIES = ["Tripping", "Slashing", "No_penalty"]
# SERIES = ["Tripping"]

In [20]:
def read_image(image_dir, frame_no):
    
    frame_name = "frame_%04d.png" % int(frame_no)
    image = imageio.imread(osp.join(image_dir, frame_name))
    image = ia.imresize_single_image(image, (360, 640))
    #         ia.imshow(image)
    
    return image
    

In [21]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

def get_players(pose):
    
    keys = set().union(*(d.keys() for d in pose))
    keys.remove('frameNum')
    all_players = list(keys)
    all_players.sort(key=natural_keys)
    
    return all_players

In [22]:
def get_keypoints(frame_no, players, pose, kps, kps_meta):
    body = pose[frame_no]
    for key in players:
        player_body = body[key]
        for i in range(0,48,3):
            x = player_body[i]
            y = player_body[i+1]
            kps.append(Keypoint(x,y))
            meta = [frame_no, key, i]
            kps_meta.append(meta)
    
    return kps, kps_meta
    
    

In [23]:
# def flip():
    
#     seq = iaa.Sequential([
#     iaa.flip.Fliplr(p=1)])
#     seq_det = seq.to_deterministic()
    
#     return seq_det

In [24]:
import random


def pers_scale(affine_scale, pers_scale):
    
    choice = random.randint(0,1)
#     print(choice)

    if choice == 0:
        #perspective
        
        print("perspective" + str(pers_scale))
        seq = iaa.Sequential([
        iaa.PerspectiveTransform(scale=(pers_scale, pers_scale))])
        seq_det = seq.to_deterministic()

    elif choice == 1:
        #scale
        print("scale" + str(affine_scale))
        seq = iaa.Sequential([
        iaa.Affine(scale=(affine_scale,affine_scale))])
        seq_det = seq.to_deterministic()

    return seq_det
    
    

In [25]:
def test_draw(frame, frame_aug, kpsoi, kpsoi_aug ):
    ia.imshow(
    np.hstack([
        kpsoi.draw_on_image(frame, size=7),
        kpsoi_aug.draw_on_image(frame_aug, size=7)
    ])
)
    

In [26]:
def Augment_2(image_dir, pose):
    keypoints_augment = []
    keypoints_meta = []
    frames_augment = np.zeros((64,360,640,3))
    
    per_scale = [0.05,0.06,0.07,0.08,0.09,0.1]
    affine_sclae = [1.2,1.3,1.4,1.5]
    
    per_rand = random.randint(0,5)
    affine_rand = random.randint(0,3)
    

    affine_augmenter = pers_scale(affine_scale=affine_sclae[affine_rand], pers_scale=per_scale[per_rand])
    
    for frame_no in range(64):
        
        kps = []  
        kps_meta = []
        frame = read_image(image_dir, frame_no)
        players = get_players(pose)
        kps, kps_meta = get_keypoints(frame_no, players, pose, kps, kps_meta)
        kpsoi = KeypointsOnImage(kps, shape=frame.shape)
        frame_aug, kpsoi_aug = affine_augmenter(image=frame, keypoints=kpsoi)
#         test_draw(frame, frame_aug, kpsoi, kpsoi_aug )
        frames_augment[frame_no,:,:,:] = frame_aug
        kpsoi_aug = [kpsoi_aug]
        keypoints_augment.append(kpsoi_aug)
        keypoints_meta.append(kps_meta)

    return keypoints_augment, frames_augment, keypoints_meta

In [27]:
# def Augment(image_dir, pose):
#     keypoints_augment = []
#     keypoints_meta = []
#     frames_augment = np.zeros((64,360,640,3))
#     flip_augmenter = flip()
    
#     for frame_no in range(64):

#         kps = []  
#         kps_meta = []
#         frame = read_image(image_dir, frame_no)
#         players = get_players(pose)
#         kps, kps_meta = get_keypoints(frame_no, players, pose, kps, kps_meta)
#         kpsoi = KeypointsOnImage(kps, shape=frame.shape) 
#         frame_aug, kpsoi_aug = flip_augmenter(image=frame, keypoints=kpsoi)
# #         test_draw(frame, frame_aug, kpsoi, kpsoi_aug )
#         frames_augment[frame_no,:,:,:] = frame_aug
#         kpsoi_aug = [kpsoi_aug]
#         keypoints_augment.append(kpsoi_aug)
#         keypoints_meta.append(kps_meta)

#     return keypoints_augment, frames_augment, keypoints_meta

In [28]:
def write_image(dest_frame_pose_path, frames_augment):
    
    for frame_no in range(frames_augment.shape[0]):
        
        image = frames_augment[frame_no,:,:,:]
        image_uint8 = image.astype(np.uint8)
        frame_name = "frame_%04d.png" % int(frame_no)
        imageio.imsave(osp.join(dest_frame_pose_path, frame_name), image_uint8)
        
    print(dest_frame_pose_path)
    

In [30]:
def write_pose(dest_frame_pose_path, keypoints_augment, keypoints_meta):
    
    json_list_of_dict = []
    flat_keypoints_augment = [item for sublist in keypoints_augment for item in sublist]
    print(dest_frame_pose_path)
    for frame_no, (pose_aug, pose_meta) in enumerate(zip(flat_keypoints_augment,keypoints_meta)):
        json_dict = {}
        json_dict["frameNum"] = frame_no
        for idx, (coord, meta) in enumerate(zip(pose_aug, pose_meta)):
            player = meta[1]
            xy = [coord.x, coord.y, 1.0]
            json_dict.setdefault(player,[]).extend(xy)
            
        json_list_of_dict.append(json_dict)
    
    name = os.path.split(dest_frame_pose_path)[-1]
    save_path = os.path.join(dest_frame_pose_path, name + '.json')
    with open(save_path, 'w') as file:
        json.dump(eval(str(json_list_of_dict)), file, ensure_ascii=False, indent=4)
            
#     print(json_list_of_dict)


In [31]:
dest_dir = "./aug_hockey_dataset2"
SERIES = ["Tripping", "Slashing", "No_penalty"]

if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)




In [32]:
# for series_no in SERIES:
#     count = 0
#     dest_sub_folder = os.path.join(dest_dir, series_no)
#     if not os.path.exists(dest_sub_folder):
#         os.makedirs(dest_sub_folder)
    
#     for video in os.listdir(osp.join(dataset_dir, series_no)):
#         with open(osp.join(dataset_dir, series_no, video, video + ".json"), 'r') as f:
#             pose =  json.load(f)
        
#         keypoints_augment, frames_augment, keypoints_meta  = Augment(osp.join(dataset_dir, series_no, video), pose)
#         count = count +1
        
#         dest_frame_pose_path = os.path.join(dest_sub_folder, video + "100")
        
#         if not os.path.exists(dest_frame_pose_path):
#             os.makedirs(dest_frame_pose_path)
        
#         write_image(dest_frame_pose_path, frames_augment)
#         write_pose(dest_frame_pose_path, keypoints_augment, keypoints_meta)
        
#     print(count)

In [33]:
####number 2

for series_no in SERIES:
    count = 0
    dest_sub_folder = os.path.join(dest_dir, series_no)
    
    if not os.path.exists(dest_sub_folder):
        os.makedirs(dest_sub_folder)

    for video in os.listdir(osp.join(dataset_dir, series_no)):
        
        chance = random.uniform(0, 1)
        print(chance)
            
        print(os.path.join(dataset_dir, series_no, video))

        with open(osp.join(dataset_dir, series_no, video, video + ".json"), 'r') as f:
            pose =  json.load(f)

#             a  = Augment_2(osp.join(dataset_dir, series_no, video), pose)
        keypoints_augment, frames_augment, keypoints_meta  = Augment_2(osp.join(dataset_dir, series_no, video), pose)
        count = count +1

        dest_frame_pose_path = os.path.join(dest_sub_folder, video + "200")

        if not os.path.exists(dest_frame_pose_path):
            os.makedirs(dest_frame_pose_path)

        write_image(dest_frame_pose_path, frames_augment)
        write_pose(dest_frame_pose_path, keypoints_augment, keypoints_meta)
            
    print(count)

0.9565565566904457
./hockey_dataset/Tripping/_2017-11-06-min-bos-home8
perspective0.07
./aug_hockey_dataset2/Tripping/_2017-11-06-min-bos-home8200
./aug_hockey_dataset2/Tripping/_2017-11-06-min-bos-home8200
0.5087368023767755
./hockey_dataset/Tripping/_2017-12-05-ana-vgk-home53
scale1.3
./aug_hockey_dataset2/Tripping/_2017-12-05-ana-vgk-home53200
./aug_hockey_dataset2/Tripping/_2017-12-05-ana-vgk-home53200
0.45290068041407705
./hockey_dataset/Tripping/_2017-11-28-chi-nsh-national407
scale1.5
./aug_hockey_dataset2/Tripping/_2017-11-28-chi-nsh-national407200
./aug_hockey_dataset2/Tripping/_2017-11-28-chi-nsh-national407200
0.8861463103747613
./hockey_dataset/Tripping/_2017-11-11-col-ott-national479
perspective0.1
./aug_hockey_dataset2/Tripping/_2017-11-11-col-ott-national479200
./aug_hockey_dataset2/Tripping/_2017-11-11-col-ott-national479200
0.5060053660753392
./hockey_dataset/Tripping/_2017-11-18-col-nsh-home215
perspective0.1
./aug_hockey_dataset2/Tripping/_2017-11-18-col-nsh-home2152

./aug_hockey_dataset2/Tripping/_2018-02-13-ana-det-home136200
./aug_hockey_dataset2/Tripping/_2018-02-13-ana-det-home136200
0.07317054590559968
./hockey_dataset/Tripping/_2017-11-04-cbj-tbl-home149
perspective0.05
./aug_hockey_dataset2/Tripping/_2017-11-04-cbj-tbl-home149200
./aug_hockey_dataset2/Tripping/_2017-11-04-cbj-tbl-home149200
0.7288043116773204
./hockey_dataset/Tripping/_2017-11-25-vgk-ari-home398
perspective0.06
./aug_hockey_dataset2/Tripping/_2017-11-25-vgk-ari-home398200
./aug_hockey_dataset2/Tripping/_2017-11-25-vgk-ari-home398200
0.24563870285926948
./hockey_dataset/Tripping/_2018-02-13-ana-det-home137
scale1.2
./aug_hockey_dataset2/Tripping/_2018-02-13-ana-det-home137200
./aug_hockey_dataset2/Tripping/_2018-02-13-ana-det-home137200
0.8580197690925824
./hockey_dataset/Tripping/_2018-02-10-col-car-home127
perspective0.07
./aug_hockey_dataset2/Tripping/_2018-02-10-col-car-home127200
./aug_hockey_dataset2/Tripping/_2018-02-10-col-car-home127200
0.9155679615239348
./hockey_d

./aug_hockey_dataset2/Tripping/_2017-11-16-phi-wpg-home549200
./aug_hockey_dataset2/Tripping/_2017-11-16-phi-wpg-home549200
0.24854987141951657
./hockey_dataset/Tripping/_2018-02-08-nyi-buf-home113
perspective0.05
./aug_hockey_dataset2/Tripping/_2018-02-08-nyi-buf-home113200
./aug_hockey_dataset2/Tripping/_2018-02-08-nyi-buf-home113200
80
0.34993805999858396
./hockey_dataset/Slashing/_2017-11-18-nyi-tbl-home218
perspective0.06
./aug_hockey_dataset2/Slashing/_2017-11-18-nyi-tbl-home218200
./aug_hockey_dataset2/Slashing/_2017-11-18-nyi-tbl-home218200
0.80776114510276
./hockey_dataset/Slashing/_2018-01-09-chi-ott-home4
scale1.2
./aug_hockey_dataset2/Slashing/_2018-01-09-chi-ott-home4200
./aug_hockey_dataset2/Slashing/_2018-01-09-chi-ott-home4200
0.7376145092202269
./hockey_dataset/Slashing/_2018-03-09-min-van-home529
scale1.4
./aug_hockey_dataset2/Slashing/_2018-03-09-min-van-home529200
./aug_hockey_dataset2/Slashing/_2018-03-09-min-van-home529200
0.6081615897026104
./hockey_dataset/Slash

./aug_hockey_dataset2/Slashing/_2018-03-05-ari-edm-home429200
./aug_hockey_dataset2/Slashing/_2018-03-05-ari-edm-home429200
0.8583518636719147
./hockey_dataset/Slashing/_2017-11-29-wpg-col-home322
perspective0.07
./aug_hockey_dataset2/Slashing/_2017-11-29-wpg-col-home322200
./aug_hockey_dataset2/Slashing/_2017-11-29-wpg-col-home322200
0.2505718858897219
./hockey_dataset/Slashing/_2018-01-07-fla-cbj-home84
perspective0.08
./aug_hockey_dataset2/Slashing/_2018-01-07-fla-cbj-home84200
./aug_hockey_dataset2/Slashing/_2018-01-07-fla-cbj-home84200
0.19220107502917927
./hockey_dataset/Slashing/_2017-11-16-ari-mtl-home517
perspective0.09
./aug_hockey_dataset2/Slashing/_2017-11-16-ari-mtl-home517200
./aug_hockey_dataset2/Slashing/_2017-11-16-ari-mtl-home517200
0.17168262628403208
./hockey_dataset/Slashing/_2017-12-07-col-tbl-home375
perspective0.08
./aug_hockey_dataset2/Slashing/_2017-12-07-col-tbl-home375200
./aug_hockey_dataset2/Slashing/_2017-12-07-col-tbl-home375200
0.8186760776140033
./hock

./aug_hockey_dataset2/No_penalty/_2018-02-05-nyr-dal-home98200
./aug_hockey_dataset2/No_penalty/_2018-02-05-nyr-dal-home98200
0.27988983835431125
./hockey_dataset/No_penalty/_2018-03-08-mtl-fla-home492
scale1.2
./aug_hockey_dataset2/No_penalty/_2018-03-08-mtl-fla-home492200
./aug_hockey_dataset2/No_penalty/_2018-03-08-mtl-fla-home492200
0.3184082206947284
./hockey_dataset/No_penalty/_2018-02-06-cgy-chi-home101
scale1.2
./aug_hockey_dataset2/No_penalty/_2018-02-06-cgy-chi-home101200
./aug_hockey_dataset2/No_penalty/_2018-02-06-cgy-chi-home101200
0.9890220895775307
./hockey_dataset/No_penalty/_2018-01-24-lak-cgy-home95
perspective0.07
./aug_hockey_dataset2/No_penalty/_2018-01-24-lak-cgy-home95200
./aug_hockey_dataset2/No_penalty/_2018-01-24-lak-cgy-home95200
0.9035657694817654
./hockey_dataset/No_penalty/_2017-11-16-pit-ott-home558
perspective0.1
./aug_hockey_dataset2/No_penalty/_2017-11-16-pit-ott-home558200
./aug_hockey_dataset2/No_penalty/_2017-11-16-pit-ott-home558200
0.4848785357931

./aug_hockey_dataset2/No_penalty/_2017-11-27-ana-chi-home289200
./aug_hockey_dataset2/No_penalty/_2017-11-27-ana-chi-home289200
0.4921127813772268
./hockey_dataset/No_penalty/_2017-11-29-tbl-bos-national279
scale1.2
./aug_hockey_dataset2/No_penalty/_2017-11-29-tbl-bos-national279200
./aug_hockey_dataset2/No_penalty/_2017-11-29-tbl-bos-national279200
0.7900969767112215
./hockey_dataset/No_penalty/_2017-11-24-lak-ari-home234
scale1.4
./aug_hockey_dataset2/No_penalty/_2017-11-24-lak-ari-home234200
./aug_hockey_dataset2/No_penalty/_2017-11-24-lak-ari-home234200
0.16114650263300856
./hockey_dataset/No_penalty/_2017-11-18-min-wsh-home376
perspective0.09
./aug_hockey_dataset2/No_penalty/_2017-11-18-min-wsh-home376200
./aug_hockey_dataset2/No_penalty/_2017-11-18-min-wsh-home376200
0.049254503545741124
./hockey_dataset/No_penalty/_2017-11-28-van-nyi-home317
scale1.3
./aug_hockey_dataset2/No_penalty/_2017-11-28-van-nyi-home317200
./aug_hockey_dataset2/No_penalty/_2017-11-28-van-nyi-home317200
0.

./aug_hockey_dataset2/No_penalty/_2018-01-18-dal-cbj-home98200
./aug_hockey_dataset2/No_penalty/_2018-01-18-dal-cbj-home98200
0.6392662398246813
./hockey_dataset/No_penalty/_2017-11-16-wsh-col-home570
perspective0.09
./aug_hockey_dataset2/No_penalty/_2017-11-16-wsh-col-home570200
./aug_hockey_dataset2/No_penalty/_2017-11-16-wsh-col-home570200
0.19241638351160684
./hockey_dataset/No_penalty/_2017-11-16-njd-tor-home535
scale1.3
./aug_hockey_dataset2/No_penalty/_2017-11-16-njd-tor-home535200
./aug_hockey_dataset2/No_penalty/_2017-11-16-njd-tor-home535200
0.06523622241877436
./hockey_dataset/No_penalty/_2017-11-16-nsh-min-home541
perspective0.07
./aug_hockey_dataset2/No_penalty/_2017-11-16-nsh-min-home541200
./aug_hockey_dataset2/No_penalty/_2017-11-16-nsh-min-home541200
0.3960037458391531
./hockey_dataset/No_penalty/_2017-11-05-col-nyi-home12
scale1.4
./aug_hockey_dataset2/No_penalty/_2017-11-05-col-nyi-home12200
./aug_hockey_dataset2/No_penalty/_2017-11-05-col-nyi-home12200
0.87498960095